# Data Extraction

## Parsing Y- axis Ticks and Y- vals 
This code parses the x and y axes (horizontal and vertical axes) detection for the y-ticks, calculates the ratio to normalize the figure contents, specifically the bar plots with respect to the y-ticks and finally connects the evaluated y-values with their correspinding legends using clustering algorithm. 
* Note that all the figures recorded have their respective dois as a unique key of identification.

In [1]:
import cv2, imutils, re, sys, math
import xlsxwriter, json, os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from matplotlib import rcParams

In [2]:
# Directory of images to run the code on
img_dir = '../AllData/processed_images/Part2'

### Using json to parse the captions and dois for each image

In [3]:
with open('../data/urldata.json') as handler:
    urldata = json.load(handler)

In [4]:
with open('../data/doi_data.json') as doihandler:
    doidata = json.load(doihandler)

In [5]:
with open('../data/ocr-image-text.json') as ocrhandler:
    images_text = json.load(ocrhandler)

In [6]:
with open('../data/aws-rekognition-output.json') as awshandler:
    bbox_text = json.load(awshandler)

### Function to detect x-axis and y-axis for a plot

In [7]:
def findMaxConsecutiveOnes(nums) -> int:
    count = maxCount = 0
    
    for i in range(len(nums)):
        if nums[i] == 1:
            count += 1
        else:
            maxCount = max(count, maxCount)
            count = 0
                
    return max(count, maxCount)

In [8]:
def detectAxes(filepath, threshold=None, debug=False):
    if filepath is None:
        return None, None
    
    if threshold is None:
        threshold = 10
    
    image = cv2.imread(filepath)
    height, width, channels = image.shape
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[:, idx] < 200) for idx in range(width)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < width:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            break
            
        start_idx += 1
           
    yaxis = (maxindex, 0, maxindex, height)
    
    if debug:
        fig, ax = plt.subplots(1, 2)

        ax[0].imshow(image)

        ax[1].plot(maxConsecutiveOnes, color = 'k')
        ax[1].axhline(y = max(maxConsecutiveOnes) - 10, color = 'r', linestyle = 'dashed')
        ax[1].axhline(y = max(maxConsecutiveOnes) + 10, color = 'r', linestyle = 'dashed')
        ax[1].vlines(x = maxindex, ymin = 0.0, ymax = maxConsecutiveOnes[maxindex], color = 'b', linewidth = 4)

        plt.show()

    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[idx, :] < 200) for idx in range(height)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < height:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            
        start_idx += 1
            
    cv2.line(image, (0, maxindex), (width, maxindex),  (255, 0, 0), 2)
    xaxis = (0, maxindex, width, maxindex)
    
    if debug:
        rcParams['figure.figsize'] = 15, 8

        fig, ax = plt.subplots(1, 1)
        ax.imshow(image, aspect = 'auto')
        
    return xaxis, yaxis

In [9]:
def cleanText(image_text):
    
    return [(text, (textx, texty, w, h)) for text, (textx, texty, w, h) in image_text if text.strip() != 'I']

In [10]:
def getProbableLabels(image, image_text, xaxis, yaxis):
    y_labels = []
    x_labels = []
    legends = []
    y_text_list = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    image_text = cleanText(image_text)
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
            
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            numbers = re.findall(r'^[+-]?\d+(?:\.\d+)?[%-]?$', text)
            if bool(numbers):
                y_labels.append((text, (textx, texty, w, h)))
            else:
                y_text_list.append((text, (textx, texty, w, h)))
            
        # To the right of y-axis and bottom of x-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == 1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            x_labels.append((text, (textx, texty, w, h)))
            
        # Top of x-axis and to the right of y-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            
            # Consider non-numeric only for legends
            legends.append((text, (textx, texty, w, h)))
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(y_labels):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(y_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    y_labels_list = maxList.copy()
    
    y_labels = []
    for text, (textx, texty, w, h) in maxList:
        y_labels.append(text)
        
    # Get the x-labels by finding the maximum
    # intersections with the sweeping line
    
    maxIntersection = 0
    maxList = []
    for i in range(y1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_labels):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
            
    x_labels_list = maxList.copy()
    
    x_text = x_labels.copy()
    x_labels = []
    hmax = 0
    
    for text, (textx, texty, w, h) in maxList:
        x_labels.append(text)
        if texty + h > hmax:
            hmax = texty + h
    
    # Get possible x-text by moving from where we
    # left off in x-labels to the complete
    # height of the image.
    maxIntersection = 0
    maxList = []
    for i in range(hmax + 1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_text):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    x_text = []
    for text, (textx, texty, w, h) in maxList:
        x_text.append(text)
    
    # Get possible legend text
    # For this, we need to search both top to
    # bottom and also from left to right.
    
    legends_and_numbers = mergeTextBoxes(legends)
    
    legends = []
    for text, (textx, texty, w, h) in legends_and_numbers:
        if not re.search(r'^([(+-]*?(\d+)?(?:\.\d+)*?[-%) ]*?)*$', text):
            legends.append((text, (textx, texty, w, h)))
    
    
    def canMerge(group, candidate):
        candText, candRect = candidate
        candx, candy, candw, candh = candRect
        
        for memText, memRect in group:
            memx, memy, memw, memh = memRect
                
            if abs(candy - memy) <= 5 and abs(candy + candh - memy - memh) <= 5:
                return True
            elif abs(candx - memx) <= 5:
                return True
                
        return False
    
    # Grouping Algorithm
    legend_groups = []
    for index, (text, rect) in enumerate(legends):
        #print("text: {0}, rect: {1}\n".format(text, rect))
        
        for groupid, group in enumerate(legend_groups):
            if canMerge(group, (text, rect)):
                group.append((text, rect))
                break
        else:
            legend_groups.append([(text, rect)])
    
    #print(legend_groups)
    #print("\n\n")
    
    maxList = []
    
    if len(legend_groups) > 0:
        maxList = max(legend_groups, key = len)

    legends = []
    for text, (textx, texty, w, h) in maxList:
        legends.append(text)
        
    return image, x_labels, x_labels_list, x_text, y_labels, y_labels_list, y_text_list, legends, maxList

In [11]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

## Getting the Ratio for y-value matching 

Similar to the label detection logic, y-ticks are detected: 

Y-ticks 

*  Check only the numerical boxes which are to the left of y-axis and to the top of x-axis.
*  Run a line sweep from left end of the image to the y-axis position, and check when the sweeping line intersects with the maximum number of numerical boxes.
* The numerical boxes are then used as bounding boxes for calculating the y-ticks.

Ratio:
* Difference between the y-ticks is then calculated.  
* Only consider the mean difference between the y-ticks, rejecting the outliers from the calculated values.
* Finally, the value-tick ratio to normalize the heights of the bounding boxes is calculated by:  

$$
ratio  = \frac{\Delta ticks_{average} }{y\;ticks_{average}}
$$


In [12]:
def getRatio(path, xaxis, yaxis):
    list_text = []
    list_ticks = []
    
    filepath = img_dir + "/" + path.name
    
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, channels = image.shape

    image_text = images_text[path.name]
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
                    
        (x1, y1, x2, y2) = xaxis
        (x11, y11, x22, y22) = yaxis
        
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            # Consider numeric only for ticks on y-axis
            numbers = re.findall(r'\d+(?:\.\d+)?', text)
            if bool(numbers):
                list_text.append((numbers[0], (textx, texty, w, h)))
                          
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(list_text):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(list_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Get list of text and ticks
    list_text = []
    for text, (textx, texty, w, h) in maxList:
        list_text.append(float(text))
        list_ticks.append(float(texty + h))
        
    text_sorted = (sorted(list_text))
    ticks_sorted  = (sorted(list_ticks))
    
    ticks_diff = ([ticks_sorted[i] - ticks_sorted[i-1] for i in range(1, len(ticks_sorted))])
    text_diff = ([text_sorted[i] - text_sorted[i-1] for i in range(1, len(text_sorted))])
    print("[get text-to-tick ratio] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    # Detected text may not be perfect! Remove the outliers.
    ticks_diff = reject_outliers(np.array(ticks_diff), m=1)
    text_diff = reject_outliers(np.array(text_diff), m=1)
    print("[reject_outliers] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    normalize_ratio = np.array(text_diff).mean() / np.array(ticks_diff).mean()

    return text_sorted, normalize_ratio

In [13]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [14]:
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) <= m * np.std(data)]

## Writing to Excel workbook

In [15]:
def addToExcel(dataname, data, row):
    col = 0

    worksheet.write(row, col, dataname)
    for content in data:
        col += 1
        worksheet.write(row, col, content)

## Matching the ratio for final data extraction 

Y-val data:
    - The height of each bounding box is recorded by the help of the merging rectangles during Cluster count 
    estimation method.
    - Eventually, we used the ratio to calculate the y-values as:

$$
y_{values}  =  ratio \times height_{bounding\;box}
$$


In [16]:
def mergeRects(contours, mode='contours'):
    rects = []
    rectsUsed = []

    # Just initialize bounding rects and set all bools to false
    for cnt in contours:
        if mode == 'contours':
            rects.append(cv2.boundingRect(cnt))
        elif mode == 'rects':
            rects.append(cnt)
        
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[0]

    rects.sort(key = getXFromRect)

    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 5
    yThr = 5

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[0]
            currxMax = supVal[0] + supVal[2]
            curryMin = supVal[1]
            curryMax = supVal[1] + supVal[3]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[0]
                candxMax = subVal[0] + subVal[2]
                candyMin = subVal[1]
                candyMax = subVal[1] + subVal[3]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)

                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])
    
    return acceptedRects

In [17]:
def mergeTextBoxes(textboxes):
    rects = []
    rectsUsed = []
    
    # Just initialize bounding rects and set all bools to false
    for box in textboxes:
        rects.append(box)
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[1][0]
    
    def getYFromRect(item):
        return item[1][1]

    rects.sort(key = lambda x: (getYFromRect, getXFromRect))
    
    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 10
    yThr = 0

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[1][0]
            currxMax = supVal[1][0] + supVal[1][2]
            curryMin = supVal[1][1]
            curryMax = supVal[1][1] + supVal[1][3]
            currText = supVal[0]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[1][0]
                candxMax = subVal[1][0] + subVal[1][2]
                candyMin = subVal[1][1]
                candyMax = subVal[1][1] + subVal[1][3]
                candText = subVal[0]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)
                    currText = currText + ' ' + candText
                    
                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currText, (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin)])
    
    return acceptedRects

In [18]:
def nearbyRectangle(current, candidate, threshold):
    (currx, curry, currw, currh) = current
    (candx, candy, candw, candh) = candidate
    
    currxmin = currx
    currymin = curry
    currxmax = currx + currw
    currymax = curry + currh
    
    candxmin = candx
    candymin = candy
    candxmax = candx + candw
    candymax = candy + candh
    
    # If candidate is on top, and is close
    if candymax <= currymin and candymax + threshold >= currymin:
        return True
    
    # If candidate is on bottom and is close
    if candymin >= currymax and currymax + threshold >= candymin:
        return True
    
    # If intersecting at the top, merge it
    if candymax >= currymin and candymin <= currymin:
        return True
    
    # If intersecting at the bottom, merge it
    if currymax >= candymin and currymin <= candymin:
        return True
    
    # If intersecting on the sides or is inside, merge it
    if (candymin >= currymin and
        candymin <= currymax and
        candymax >= currymin and
        candymax <= currymax):
        return True
    
    return False

In [19]:
def euclidean(v1, v2):
    return sum((p - q) ** 2 for p, q in zip(v1, v2)) ** .5

def angle_between(p1, p2):
    
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]

    return math.atan2(deltaY, deltaX) / math.pi * 180
    
def RectDist(rectA, rectB):
    (rectAx, rectAy, rectAw, rectAh) = rectA
    (rectBx, rectBy, rectBw, rectBh) = rectB
    
    return abs(rectAx + rectAx / 2 - rectBx - rectBx / 2)

In [20]:
def getProbableYLabels(image, contours, xaxis, yaxis):
    y_labels = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, rect in enumerate(contours):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(contours[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
                    
    return image, maxList

In [21]:
def expand(points, margin):
    return np.array([
        [[points[0][0][0] - margin, points[0][0][1] - margin]],
        [[points[1][0][0] + margin, points[1][0][1] - margin]],
        [[points[2][0][0] + margin, points[2][0][1] + margin]],
        [[points[3][0][0] - margin, points[3][0][1] + margin]]])

## Saving y-values in our data excel sheet 

In [22]:
images = []
texts = []

def getYVal(img_dir):
    yValueDict = {}
    for index, path in enumerate(Path(img_dir).iterdir()):
        if path.name.endswith('.png') or path.name.endswith('.jpg'):
            filepath = img_dir + "/" + path.name
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_height, img_width, _ = img.shape
            
            # Axes detection
            xaxis, yaxis = detectAxes(filepath)
            
            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)

            image_text = images_text[path.name]
            img, x_labels, x_labels_list, _, _, _, _, legends, legendBoxes = getProbableLabels(img, image_text, xaxis, yaxis)
            actual_image = img.copy()
            
            try:
                list_text, normalize_ratio = getRatio(path, xaxis, yaxis)
                print("[getYVal] legends: {0}".format(legends))
                print("[{0}] path: {1}, ratio: {2}".format(index, path.name, normalize_ratio), end='\n\n')
            
                texts = bbox_text[path.name]['TextDetections']
                
                for text in texts:
                    if text['Type'] == 'WORD' and text['Confidence'] >= 80:
                        vertices = [[vertex['X'] * img_width, vertex['Y'] * img_height] for vertex in text['Geometry']['Polygon']]
                        vertices = np.array(vertices, np.int32)
                        vertices = vertices.reshape((-1, 1, 2))

                        img = cv2.fillPoly(img, [expand(vertices, 1)], (255, 255, 255))

                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                threshold = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
                
                contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                contours = [contour for contour in contours if cv2.contourArea(contour) < 0.01 * img_height * img_width]

                contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]
                rects = [cv2.boundingRect(contour) for contour in contours]

                groups = []
                legendtexts = []
                legendrects = []
    
                for box in legendBoxes:
                    text, (textx, texty, width, height) = box
                    bboxes = filterBbox(rects, box)
                    
                    if bboxes is not None:
                        for rect in [bboxes]:
                            (x, y, w, h) = rect
                            legendrects.append(rect)
                            
                            group = boxGroup(actual_image, rect)[0]
                            group = [arr.tolist() for arr in group]
                            
                            groups.append(group)
                            legendtexts.append(text)
                            
                            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)

                        cv2.rectangle(img, (textx, texty), (textx + width, texty + height), (255, 0, 0), 2)
                                 
                data = {}
                for legend in legends:
                    data[legend] = {}
                    
                    for x_label, box in x_labels_list:
                        data[legend][x_label] = 0.0
                        
                for i in range(len(groups)):

                    img = cv2.imread(img_dir + "/" + path.name)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    legendtext = legendtexts[i]
                    
                    for box in legendrects:
                        (textx, texty, width, height) = box
                        cv2.rectangle(img, (textx, texty), (textx + width, texty + height), (255, 255, 255), cv2.FILLED)
                    
                    mask = None
                    for value in groups[i]:
                        COLOR_MIN = np.array([value[0], value[1], value[2]], np.uint8)
                        COLOR_MAX = np.array([value[0], value[1], value[2]], np.uint8)

                        if mask is None:
                            mask = cv2.inRange(img, COLOR_MIN, COLOR_MAX)
                        else:
                            mask = mask | cv2.inRange(img, COLOR_MIN, COLOR_MAX)

                    image = cv2.bitwise_and(img, img, mask = mask)
                    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, (3, 3))

                    edged = cv2.Canny(image, 0, 250)
                    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    contours = [contour for contour in contours if cv2.contourArea(contour) > 0.]

                    # Remove noisy ones!
                    if len(contours) == 0 or len(contours) > 100:
                        continue

                    contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]

                    rects = mergeRects(contours)
                    textBoxes = []
                    labels = []
                    
                    for rectBox in rects:
                        min_distance = sys.maxsize
                        closestBox = None
                        labeltext = None

                        for text, textBox in x_labels_list:
                            if RectDist(rectBox, textBox) < min_distance:
                                closestBox = textBox
                                min_distance = RectDist(rectBox, textBox)
                                labeltext = text

                        textBoxes.append(closestBox)
                        labels.append(labeltext)
                        
                    list_len = []
                    
                    for rect in rects:
                        list_len.append((rect, float(rect[3])))

                    # y-values will be a product of the normalize ratio and each length              
                    y_val = [(rect, round(l* normalize_ratio, 1)) for rect, l in list_len]
                    
                    for x_label, box in x_labels_list:
                        (x, y, w, h) = box
                        value = 0.0
                        closest = None
                        dist = sys.maxsize
                        
                        for index, item in enumerate(y_val):
                            if labels[index] == x_label:
                                (vx, vy, vw, vh) = item[0]
                                if abs(x + w/2 - vx - vw/2) < dist:
                                    dist = abs(x + w/2 - vx - vw/2)
                                    closest = item[0]
                                    value = item[1]
                             
                        data[legendtext][x_label] = value
                     
                yValueDict[path.name] = data
            
            except Exception as e:
                print(e)
                continue
                
    return yValueDict

In [23]:
def filterBbox(rects, legendBox):
    text, (textx, texty, width, height) = legendBox
    
    filtered = []
    for rect in rects:
        (x, y, w, h) = rect
        if abs(y - texty) <= 10 and abs(y - texty + h - height) <= 10:
            filtered.append(rect)

    filtered = mergeRects(filtered, 'rects')
    
    closest = None
    dist = sys.maxsize
    for rect in filtered:
        (x, y, w, h) = rect
        if abs(x + w - textx) <= dist:
            dist = abs(x + w - textx)
            closest = rect
    
    return closest

In [24]:
def boxGroup(img, box):
    (x, y, w, h) = box

    image = img[y:y+h, x:x+w].reshape((h * w, 3))
    values, counts = np.unique(image, axis = 0, return_counts = True)

    # Remove white and near-by pixels
    threshold = 5
    for r in range(255 - threshold, 256):
        for g in range(255 - threshold, 256):
            for b in range(255 - threshold, 256):
                image = image[np.where((image != [r, g, b]).any(axis = 1))]

    values, counts = np.unique(image, axis = 0, return_counts = True)
                
    sort_indices = np.argsort(-counts)
    values, counts = values[sort_indices], counts[sort_indices]

    groups = []
    groupcounts = []

    for idx, value in enumerate(values):
        grouped = False

        for groupid, group in enumerate(groups):
            for member in group:
                r, g, b = member
                vr, vg, vb = value

                if (abs(vr.astype(np.int16) - r.astype(np.int16)) <= 5 and
                    abs(vg.astype(np.int16) - g.astype(np.int16)) <= 5 and
                    abs(vb.astype(np.int16) - b.astype(np.int16)) <= 5):
                        group.append(value)
                        groupcounts[groupid] += counts[idx]
                        grouped = True
                        break

            if grouped:
                break

        if not grouped:
            groups.append([value])
            groupcounts.append(counts[idx])

    groupcounts = np.array(groupcounts)
    sort_indices = np.argsort(-groupcounts)
    new_groups = [groups[i] for i in sort_indices]
    groups = new_groups
    
    return groups

In [25]:
yValueDict = getYVal(img_dir)

[get text-to-tick ratio] ticks_diff: [44.0, 45.0, 45.0, 46.0, 46.0, 44.0], text_diff: [15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
[reject_outliers] ticks_diff: [45. 45.], text_diff: [15. 15. 15. 15. 15. 15.]
[getYVal] legends: ['Aqueous Phase', 'Gaseous Phase', 'Solid Residue', 'Biocrude Oil', 'CH', 'H 4']
[0] path: 08813216-Figure1-1.png, ratio: 0.3333333333333333

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['A', 'B']
[1] path: 08813216-Figure4-1.png, ratio: nan



c:\users\dsand\appdata\local\programs\python\python38\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\dsand\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\dsand\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:216: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\users\dsand\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
c:\users\dsand\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-12-6

[get text-to-tick ratio] ticks_diff: [34.0, 34.0, 33.0, 34.0, 35.0, 34.0, 33.0, 35.0, 33.0], text_diff: [0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [34. 34. 34. 34.], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[getYVal] legends: ['SR', 'HSP', 'MSP', 'WSP']
[2] path: 1-s2.0-S0009250913008336-main-Figure10-1.png, ratio: 0.0029411764705882353

[get text-to-tick ratio] ticks_diff: [33.0, 34.0, 31.0, 34.0, 34.0, 34.0, 31.0, 35.0, 32.0], text_diff: [0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [33. 34. 34. 34. 34. 32.], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[getYVal] legends: ['SR', 'HSP', 'MSP', 'WSP']
[3] path: 1-s2.0-S0009250913008336-main-Figure11-1.png, ratio: 0.002

[get text-to-tick ratio] ticks_diff: [71.0, 71.0, 70.0, 72.0, 70.0, 71.0], text_diff: [0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009]
[reject_outliers] ticks_diff: [71. 71. 71.], text_diff: [0.1 0.1 0.1 0.1]
[getYVal] legends: ['Fuel Specifications:', 'Gasoline:', '0.720-0.775 g/ml', '0.775-0.840 g/ml', 'Diesel:', '.820-0.840 g/mI Marine RMA', '<0.920 g/ml', 'Jet']
[24] path: 1-s2.0-S0016236115011084-main-Figure7-1.png, ratio: 0.0014084507042253518

[get text-to-tick ratio] ticks_diff: [65.0, 189.0], text_diff: [60.0, 20.0]
[reject_outliers] ticks_diff: [ 65. 189.], text_diff: [60. 20.]
[getYVal] legends: ['Conversion', 'Yield of paraffin']
[25] path: 1-s2.0-S0016236115011345-main-Figure2-1.png, ratio: 0.31496062992125984

[get text-to-tick ratio] ticks_diff: [70.0, 70.0, 72.0, 69.0], text_diff: [10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [70. 70.], text_diff: [10. 10. 10. 10.]
[getYVal] legends: 

[get text-to-tick ratio] ticks_diff: [43.0, 41.0, 42.0, 43.0, 42.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [42. 42.], text_diff: [10. 10. 10. 10. 10.]
[getYVal] legends: ['260 C.', "280'C.", '300C']
[47] path: 1-s2.0-S0016236118303363-main-Figure5-1.png, ratio: 0.23809523809523808

[get text-to-tick ratio] ticks_diff: [55.0, 54.0, 55.0, 55.0, 57.0, 53.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [55. 54. 55. 55.], text_diff: [10. 10. 10. 10. 10. 10.]
[getYVal] legends: []
[48] path: 1-s2.0-S0016236118304496-main-Figure4-1.png, ratio: 0.182648401826484

[get text-to-tick ratio] ticks_diff: [51.0, 50.0, 51.0, 51.0, 52.0, 49.0, 51.0, 51.0, 50.0], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [51. 50. 51. 51. 51. 51. 50.], text_diff: [5. 5. 5. 5. 5. 5. 5. 5. 5.]
[getYVal] legends: []
[49] path: 1-s2.0-S0016236118304496-main-Figure5-1.png, ratio: 0.09859154929577464

[get text-to-

...

[get text-to-tick ratio] ticks_diff: [33.0, 29.0, 36.0, 31.0, 33.0], text_diff: [6.0, 10.0, 10.0, 20.0, 10.0]
[reject_outliers] ticks_diff: [33. 31. 33.], text_diff: [10. 10. 10.]
[getYVal] legends: []
[1127] path: ef900379p-Figure8-1.png, ratio: 0.3092783505154639

[get text-to-tick ratio] ticks_diff: [54.0, 53.0, 52.0, 52.0], text_diff: [8.0, 8.0, 8.0, 8.0]
[reject_outliers] ticks_diff: [53. 52. 52.], text_diff: [8. 8. 8. 8.]
[getYVal] legends: ['Calorific value', 'Moisture content']
[1128] path: Effect of hydration calcination CaO on the deoxygenation of bio oil from pyrolysis of Nannochloropsis sp-Figure4-1.png, ratio: 0.15286624203821655

[get text-to-tick ratio] ticks_diff: [33.0, 32.0, 31.0, 31.0, 64.0, 31.0, 31.0], text_diff: [2.0, 2.0, 4.0, 2.0, 2.0, 2.0, 2.0]
[reject_outliers] ticks_diff: [33. 32. 31. 31. 31. 31.], text_diff: [2. 2. 2. 2. 2. 2.]
[getYVal] legends: []
[1129] path: energies-09-00888-Figure1-1.png, ratio: 0.06349206349206349

[get text-to-tick ratio] ticks_diff:

[get text-to-tick ratio] ticks_diff: [32.0, 32.0, 32.0, 32.0, 33.0, 33.0, 31.0], text_diff: [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
[reject_outliers] ticks_diff: [32. 32. 32. 32.], text_diff: [50. 50. 50. 50. 50. 50. 50.]
[getYVal] legends: []
[1152] path: ep.12937-Figure11-1.png, ratio: 1.5625

[get text-to-tick ratio] ticks_diff: [50.0, 52.0, 5.0, 44.0, 52.0], text_diff: [0.0, 15.0, 4985.0, 15000.0, 5000.0]
[reject_outliers] ticks_diff: [50. 52. 44. 52.], text_diff: [   0.   15. 4985. 5000.]
[getYVal] legends: []
[1153] path: ep.12937-Figure12-1.png, ratio: 50.505050505050505

[get text-to-tick ratio] ticks_diff: [55.0, 54.0, 56.0, 56.0, 56.0], text_diff: [0.02, 0.019999999999999997, 0.020000000000000004, 0.020000000000000004, 0.01999999999999999]
[reject_outliers] ticks_diff: [55. 56. 56. 56.], text_diff: [0.02 0.02]
[getYVal] legends: []
[1154] path: ep.12937-Figure13-1.png, ratio: 0.000358744394618834

[get text-to-tick ratio] ticks_diff: [60.0, 61.0, 58.0, 60.0], text_diff: [2

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1180] path: ie202385s-Figure3-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1181] path: ie5020684-Figure2-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1182] path: ie5020684-Figure3-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [76.0, 75.0], text_diff: [5.0, 5.0]
[reject_outliers] ticks_diff: [76. 75.], text_diff: [5. 5.]
[getYVal] legends: ['CH no catalyst', 'C,He, no catalyst', 'CH, two-chamber', 'C.H two-chamber 2', 'CHo one-chamber', 'C,H6 one-chamber 2']
[1183] path: ie5020684-Figure4-1.png, ratio: 0.06622516556291391

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1184] path

[get text-to-tick ratio] ticks_diff: [51.0, 51.0, 51.0, 53.0, 51.0, 52.0], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [51. 51. 51. 51. 52.], text_diff: [5. 5. 5. 5. 5. 5.]
[getYVal] legends: []
[1208] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.11-1.png, ratio: 0.09765625

[get text-to-tick ratio] ticks_diff: [63.0, 62.0, 62.0, 63.0, 61.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [62. 62.], text_diff: [10. 10. 10. 10. 10.]
[getYVal] legends: ['No Catalyst']
[1209] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.8-1.png, ratio: 0.16129032258064516

[get text-to-tick ratio] ticks_diff: [89.0, 90.0], text_diff: [4.0, 4.0]
[reject_outliers] ticks_diff: [89. 90.], text_diff: [4. 4.]
[getYVal] legends: ['lul']
[1210] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.9-1.png, ratio: 0.0446927374301676

[get text-to-tick ratio] ticks_diff: [40.0, 41.0, 40.0, 41.0, 79.0], text

[get text-to-tick ratio] ticks_diff: [128.0, 66.0, 64.0], text_diff: [10.0, 10.0, 20.0]
[reject_outliers] ticks_diff: [66. 64.], text_diff: [10. 10.]
[getYVal] legends: ['Pine Wood', 'THERMAL', 'KOH', 'K K,CO,']
[1234] path: Singh2015_Article_HydrothermalLiquefactionOfAgri-Figure6-1.png, ratio: 0.15384615384615385

[get text-to-tick ratio] ticks_diff: [70.0, 72.0, 72.0], text_diff: [10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [72. 72.], text_diff: [10. 10. 10.]
[getYVal] legends: ['DEODAR', 'THERMAL', 'KOH', 'K,CO,']
[1235] path: Singh2015_Article_HydrothermalLiquefactionOfAgri-Figure7-1.png, ratio: 0.1388888888888889

[get text-to-tick ratio] ticks_diff: [128.0, 128.0], text_diff: [20.0, 20.0]
[reject_outliers] ticks_diff: [128. 128.], text_diff: [20. 20.]
[getYVal] legends: ['Deodar', 'THERMAL', 'KOH', 'K,CO,']
[1236] path: Singh2015_Article_HydrothermalLiquefactionOfAgri-Figure8-1.png, ratio: 0.15625

[get text-to-tick ratio] ticks_diff: [64.0, 62.0, 65.0, 62.0], text_diff: [10.

In [26]:
workbook = xlsxwriter.Workbook('../results/FigureData1.xlsx', {'nan_inf_to_errors': True})

for index, path in enumerate(Path(img_dir).iterdir()):
    if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg'):
        if path.name in yValueDict:
            filepath = img_dir + "/" + path.name
            print("[{0}] path: {1}".format(index, path.name))
            
            image = cv2.imread(filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            height, width, channels = image.shape
            xaxis, yaxis = detectAxes(filepath)
            y_text = []

            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)
                
            image_text = images_text[path.name]
            image, x_labels, _, x_text, y_labels, y_labels_list, y_text_list, legends, _ = getProbableLabels(image,
                                                                                                  image_text,
                                                                                                  xaxis,
                                                                                                  yaxis)
            
            # Sort bounding rects by y coordinate
            def getYFromRect(item):
                return item[1][1]

            y_labels_list.sort(key = getYFromRect)
            y_text_list.sort(key = getYFromRect, reverse=True)
            
            for text, (textx, texty, w, h) in y_text_list:
                y_text.append(text)

            # Append doi values for each image.
            pdfname = '-'.join(path.name.split('-')[:-2]) + '.pdf'
            
            if pdfname in doidata['Doi']:
                doi = doidata['Doi'][pdfname]
            else:
                doi = ''
             
            # Write to Excel
            worksheet = workbook.add_worksheet()            
            
            addToExcel("doi", doi, 0)
            addToExcel("file name", [path.name], 1)
            addToExcel("x-text", x_text, 2)
            addToExcel("x-labels", x_labels, 3)
            addToExcel("y-text", y_text, 4)
            addToExcel("y-labels", y_labels, 5)
            addToExcel("legends", legends, 6)
            
            data = yValueDict[path.name]

            column = 9
            for legend, datadict in data.items():
                if column == 9:
                    addToExcel("", datadict.keys(), 8)    
                    
                addToExcel(legend, datadict.values(), column)
                column += 1
            
            # Print the output here!
            print("file name    :  ", path.name)
            print("doi          :  ", doi)
            print("x-text       :  ", x_text)
            print("x-labels     :  ", x_labels)
            print("y-text       :  ", y_text)
            print("y-labels     :  ", y_labels)
            print("legends      :  ", legends)
            print("data         :  ", data, end= "\n\n")
            
            # Insert the image
            worksheet.insert_image('J21', filepath)

# Close the excel workbook!
workbook.close()

[0] path: 08813216-Figure1-1.png
file name    :   08813216-Figure1-1.png
doi          :   []
x-text       :   []
x-labels     :   ['270 0', "290'0", '310 C', "330'C", '350']
y-text       :   ['perce', 'ntage', 'of', 'products', '(%)']
y-labels     :   ['105', '90-', '75', '60', '45', '30', '15']
legends      :   ['Aqueous Phase', 'Gaseous Phase', 'Solid Residue', 'Biocrude Oil', 'CH', 'H 4']
data         :   {'Aqueous Phase': {'270 0': 1.0, "290'0": 33.7, '310 C': 28.7, "330'C": 30.3, '350': 36.7}, 'Gaseous Phase': {'270 0': 17.0, "290'0": 20.7, '310 C': 25.7, "330'C": 20.0, '350': 19.3}, 'Solid Residue': {'270 0': 21.0, "290'0": 18.7, '310 C': 17.7, "330'C": 18.3, '350': 15.3}, 'Biocrude Oil': {'270 0': 26.7, "290'0": 25.0, '310 C': 22.0, "330'C": 30.0, '350': 27.0}, 'CH': {'270 0': 2.7, "290'0": 3.7, '310 C': 4.3, "330'C": 4.7, '350': 5.0}, 'H 4': {'270 0': 0.0, "290'0": 0.0, '310 C': 0.0, "330'C": 0.7, '350': 0.0}}

[1] path: 08813216-Figure4-1.png
file name    :   08813216-Figure4-

file name    :   1-s2.0-S0016236114000477-main-Figure5-1.png
doi          :   ['doi:10.1016/j.fuel.2014.01.037']
x-text       :   ['ZrO2', 'ZrO2']
x-labels     :   ['K2C03', 'K2C03', 'K2C03', 'K2C03', 'Gold', 'Gold']
y-text       :   ['CIT', 'ar', 'content']
y-labels     :   ['20', '18', '16', '14', '12', '2', '10']
legends      :   ['A', 'con: 0.12', '.sh', 'tent', 'wt%']
data         :   {'A': {'K2C03': 0.0, 'Gold': 0.0}, 'con: 0.12': {'K2C03': 0.0, 'Gold': 0.0}, '.sh': {'K2C03': 0.0, 'Gold': 0.0}, 'tent': {'K2C03': 0.0, 'Gold': 0.0}, 'wt%': {'K2C03': 0.0, 'Gold': 0.0}}

[14] path: 1-s2.0-S0016236114000477-main-Figure8-1.png
file name    :   1-s2.0-S0016236114000477-main-Figure8-1.png
doi          :   ['doi:10.1016/j.fuel.2014.01.037']
x-text       :   ['Gold', 'Gold', 'ZrO2']
x-labels     :   ['K2CO3', 'K2CO3', 'K2C03', 'K2C03']
y-text       :   ['Fatty', 'acid', 'content', 'lwt%l']
y-labels     :   ['14', '12', '10', '8', '4', '2']
legends      :   ['C-16:0', 'C-18:0. 18:1, 18:2', 

file name    :   1-s2.0-S0016236115011084-main-Figure6-1.png
doi          :   ['doi:10.1016/j.fuel.2015.10.094']
x-text       :   ['HHV.', 'O.', 'H.']
x-labels     :   ['10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
y-text       :   ['C [wt.-%]', 'H', '[wt.-%]', '[wt.-%', 'H/C', '[mol/mol]', 'O/C', '[mol/mol]', 'H/C eff', 'HHV', '[MJ', 'kg-1]']
y-labels     :   ['0']
legends      :   ['Dist. Balance 40.43', 'Biocrude 1.37', 'Error [%)']
data         :   {'Dist. Balance 40.43': {'10': 0.0, '20': 0.0, '30': 0.0, '40': 0.0, '50': 0.0, '60': 0.0, '70': 0.0, '80': 0.0, '90': 0.0, '100': 0.0}, 'Biocrude 1.37': {'10': 0.0, '20': 0.0, '30': 0.0, '40': 0.0, '50': 0.0, '60': 0.0, '70': 0.0, '80': 0.0, '90': 0.0, '100': 0.0}, 'Error [%)': {'10': 0.0, '20': 0.0, '30': 0.0, '40': 0.0, '50': 0.0, '60': 0.0, '70': 0.0, '80': 0.0, '90': 0.0, '100': 0.0}}

[24] path: 1-s2.0-S0016236115011084-main-Figure7-1.png
file name    :   1-s2.0-S0016236115011084-main-Figure7-1.png
doi          :   [

file name    :   1-s2.0-S0016236116311632-main-Figure5-1.png
doi          :   ['doi:10.1016/j.fuel.2016.11.059']
x-text       :   ['Distillate', 'fractions']
x-labels     :   ['gasoline', 'diesel', 'kerosene', 'VGO', 'residue']
y-text       :   ['O', 'Content/%']
y-labels     :   ['90', '80', '70', '60', '50', '40', '30', '20', '10']
legends      :   ['nitrogen', 'fraction']
data         :   {'nitrogen': {'gasoline': 4.3, 'diesel': 3.2, 'kerosene': 7.5, 'VGO': 11.9, 'residue': 2.0}, 'fraction': {'gasoline': 0.0, 'diesel': 0.0, 'kerosene': 0.0, 'VGO': 0.0, 'residue': 0.0}}

[33] path: 1-s2.0-S0016236117301709-main-Figure1-1.png
file name    :   1-s2.0-S0016236117301709-main-Figure1-1.png
doi          :   ['doi:10.1016/j.fuel.2017.02.022']
x-text       :   []
x-labels     :   ['loss', 'Ash', 'Lipid', 'Protein', 'Carbohydrates']
y-text       :   ['comp', '(wt.%)']
y-labels     :   ['50', '45', '40', '35', '30', '25', '20', '15', '10', '5', '0']
legends      :   []
data         :   {}

[34

file name    :   1-s2.0-S0016236117315831-main-Figure6-1.png
doi          :   ['doi:10.1016/j.fuel.2017.12.016']
x-text       :   ['APPI', 'Heteroatom', 'Class']
x-labels     :   ["'o'n", "o'n", "fo'n", "'o'n", "SO'N", "o'n", "o'n", 'N3O3', 'N3O5', "9o'n", 'N', "9o'n", "eo'n", 'N3O2']
y-text       :   ['%', 'Relative', 'Abundance']
y-labels     :   ['8', '7', '6', '5', '4', '3', '2', '1']
legends      :   ['Total 75:25 Pine:algae', 'New in 75:25 Pine:Algae', 'Total 50:50 Pine:Algae']
data         :   {'Total 75:25 Pine:algae': {"'o'n": 5.6, "o'n": 0.4, "fo'n": 7.3, "SO'N": 3.6, 'N3O3': 0.1, 'N3O5': 1.0, "9o'n": 1.3, 'N': 1.9, "eo'n": 0.4, 'N3O2': 0.0}, 'New in 75:25 Pine:Algae': {"'o'n": 0.0, "o'n": 0.0, "fo'n": 0.0, "SO'N": 0.0, 'N3O3': 0.0, 'N3O5': 0.0, "9o'n": 0.0, 'N': 0.0, "eo'n": 0.0, 'N3O2': 0.0}, 'Total 50:50 Pine:Algae': {"'o'n": 0.0, "o'n": 0.0, "fo'n": 0.0, "SO'N": 0.0, 'N3O3': 0.0, 'N3O5': 0.0, "9o'n": 0.0, 'N': 0.0, "eo'n": 0.0, 'N3O2': 0.0}}

[46] path: 1-s2.0-S0016236118

...

file name    :   Kumar-Pant2016_Article_BiorefinerySolidCakeWasteToBio-Figure2-1.png
doi          :   ['10.1007/s13399-015-0175-5']
x-text       :   []
x-labels     :   ['Standard', 'size', 'effect']
y-text       :   ['Temperature', 'Time', 'Pressure', 'to', 'ratio', 'Water', 'biomass', 'pressure', 'Initial', 'Heating', 'rate', 'Particle', 'size']
y-labels     :   []
legends      :   ['p=0.05']
data         :   {'p=0.05': {'Standard': 0.0, 'size': 0.0, 'effect': 0.0}}

[1195] path: Life cycle analysis of low temperature hydrothermal treatment pathway to produce biodiesel from microalgae-Figure4-1.png
file name    :   Life cycle analysis of low temperature hydrothermal treatment pathway to produce biodiesel from microalgae-Figure4-1.png
doi          :   ['doi:10.1080/17597269.2016.1221297']
x-text       :   ['Case', 'study']
x-labels     :   ['2', '3', '4', '5', '7']
y-text       :   ['Net', 'energy', 'ratio', '(NER)']
y-labels     :   ['3']
legends      :   []
data         :   {}

[119

file name    :   Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.11-1.png
doi          :   ['10.1007/978-3-642-54458-3_10,']
x-text       :   ['773K', '873K', 'sludge,', '873K', 'starch.', '923K', 'starch.', '983K']
x-labels     :   ['18', 'wt%', 'Glucose,', '22', 'wt%o', 'Glucose,', '3', 'wt%', 'Sewage', '10', 'wt%', 'Corn', '12', 'wt%o', 'Potato']
y-text       :   ['Hyar', 'uc.,', 'Yie', "'PI-", 'mon']
y-labels     :   ['35', '30', '25', '20', '15', '10', '5']
legends      :   []
data         :   {}

[1209] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.8-1.png
file name    :   Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.8-1.png
doi          :   ['10.1007/978-3-642-54458-3_10,']
x-text       :   ['residue', 'residue', 'Acorn', 'ahell', 'stalk']
x-labels     :   ['Acorn', 'Tomato', 'Extracted', 'Hazelnut', 'Corncob', 'Sunflower']
y-text       :   ['dl']
y-labels     :   ['60', '50', '40', '30', '20', '10']
legends      :   ['No Catalys

file name    :   Pruksasri-Novalin2018_Article_AComprehensiveProcessingOfRice-Figure9-1.png
doi          :   ['doi:10.1007/s12649-017-9905-1']
x-text       :   ['Springer']
x-labels     :   ['pH2', 'pH4', 'pH6']
y-text       :   ['(kg', 'wate', 'N', 'rice', 'Accumulated', 'accum', 'bran)', 'water', 'ulated', 'usage', 'disso', 'Ived']
y-labels     :   ['60', '50', '40', '30', '10']
legends      :   ['primary extraction', '1st cascade', '2nd cascade', '3rd cascade (Total water', 'consumption)']
data         :   {'primary extraction': {'pH2': 0.0, 'pH4': 0.0, 'pH6': 0.0}, '1st cascade': {'pH2': 0.2, 'pH4': 0.0, 'pH6': 0.9}, '2nd cascade': {'pH2': 0.0, 'pH4': 0.0, 'pH6': 0.0}, '3rd cascade (Total water': {'pH2': 0.0, 'pH4': 0.0, 'pH6': 0.0}, 'consumption)': {'pH2': 0.0, 'pH4': 0.0, 'pH6': 0.0}}

[1220] path: s12866-017-1144-x-Figure4-1.png
file name    :   s12866-017-1144-x-Figure4-1.png
doi          :   ['doi:10.1186/s12866-017-1144-x']
x-text       :   ['To theri', 'indal.', 'eochtoo']
x

file name    :   Singh2015_Article_HydrothermalLiquefactionOfAgri-Figure6-1.png
doi          :   ['doi:10.1007/s10163-014-0277-3']
x-text       :   ['Chemical', 'Shift']
x-labels     :   ['5-28', 'ppm', '28-55', 'ppm', '55-95', 'ppm', '95-165', 'ppm', '165-180', 'ppm', '180-215', 'ppm']
y-text       :   ['Carbon']
y-labels     :   ['50', '10', '30', '20']
legends      :   ['Pine Wood', 'THERMAL', 'KOH', 'K K,CO,']
data         :   {'Pine Wood': {'5-28': 0.0, 'ppm': 0.0, '28-55': 0.0, '55-95': 0.0, '95-165': 0.0, '165-180': 0.0, '180-215': 0.0}, 'THERMAL': {'5-28': 0.0, 'ppm': 0.0, '28-55': 0.0, '55-95': 0.0, '95-165': 0.0, '165-180': 0.0, '180-215': 0.0}, 'KOH': {'5-28': 0.0, 'ppm': 0.0, '28-55': 0.0, '55-95': 0.0, '95-165': 0.0, '165-180': 0.0, '180-215': 0.0}, 'K K,CO,': {'5-28': 0.0, 'ppm': 0.0, '28-55': 0.0, '55-95': 0.0, '95-165': 0.0, '165-180': 0.0, '180-215': 0.0}}

[1235] path: Singh2015_Article_HydrothermalLiquefactionOfAgri-Figure7-1.png
file name    :   Singh2015_Article_Hy

file name    :   Sugano2008_Article_HydrothermalLiquefactionOfPlan-Figure5-1.png
doi          :   ['10.1007/s10853-007-2106-8']
x-text       :   ['Yield', '(wt%', 'd.a.f.', 'basis)']
x-labels     :   ['10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
y-text       :   ["150'C", "200'C", '250 C', '300 C', '350 C']
y-labels     :   []
legends      :   ['Residue', 'Oil', 'WS', 'Water', 'Gas']
data         :   {'Residue': {'10': 0.0, '20': 0.0, '30': 0.0, '40': 0.0, '50': 0.0, '60': 0.0, '70': 0.0, '80': 0.0, '90': 0.0, '100': 0.0}, 'Oil': {'10': 0.0, '20': 0.0, '30': 0.0, '40': 0.0, '50': 0.0, '60': 0.0, '70': 0.0, '80': 0.0, '90': 0.0, '100': 0.0}, 'WS': {'10': 0.0, '20': 0.0, '30': 0.0, '40': 0.0, '50': 0.0, '60': 0.0, '70': 0.0, '80': 0.0, '90': 0.0, '100': 0.0}, 'Water': {'10': 0.0, '20': 0.9, '30': 3.7, '40': 0.9, '50': 5.5, '60': 0.9, '70': 20.1, '80': 0.9, '90': 20.1, '100': 19.2}, 'Gas': {'10': 0.0, '20': 0.0, '30': 0.0, '40': 0.0, '50': 0.0, '60': 0.0, '70': 0.0, '80': 